# **Lab 1: Matrix Algorithms**
**Anders Ågren Thuné**

# **Abstract**

This report presents how a number of fundamental matrix and vector operations were
implemented in the Python programming language. In addition to the standard inner
product, matrix-vector product and matrix-matrix product, a CRS sparse matrix
structure was implemented along with a sparse matrix-vector product. The operations
were implemented according to mathematical descriptions and pseudo-code algorithms,
and verified using test data to ensure their correctness. All tests succeeded without
any significant surprises.

# **About the code**

In [1]:
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019
# Anders Ågren Thuné (athune@kth.se)
# Johan Hoffman (jhoffman@kth.se)

# Code written by Anders Ågren Thuné based on the template by Johan Hoffman.

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [2]:
# Load neccessary modules.

import numpy as np
from numpy import random
from math import sin, cos, pi

# **Introduction**

Matrix algorithms are central to many methods in scientific computing. Even basic
concepts such as matrix multiplication are rich enough to develop a wide range of
algorithmic skills. Additionally, structural properties in different classes of
matrices can often be utilized to derive more efficient methods
(Golub and Van Loan, 2013).

In this report, a number of basic matrix and vector operations are implemented.
The implementations are based on the lecture notes *Introduction to Scientific
Computing* (Hoffman, 2018), which both provide concise explanations and pseudo-code
algorithms to many of the central concepts. The programming language used
is Python, which has good support for vector and matrix types and a syntax
that lends itself to mathematical expressions. The following three standard
(dense) matrix operations are implemented based on the following definitions
(as described in Chapter 1.1, 2.1 and 2.1 of the lecture notes, respectively):
- The inner product $(\cdot,\cdot)$ induced by the Euclidean $l_2$-norm in
$\mathbb{R}^n$, defined as $(x,y) = {\displaystyle \sum_{i=1}^{n}x_iy_i}$
- Matrix-vector multiplication $Ax = b$, defined as
$Ax={\displaystyle\sum_{j=1}^{n}x_ja_j}$, where $a_j$ is the $j$th column of $a$.
- Matrix-matrix multiplication $B = AC$, defined by
$b_{ij}= {\displaystyle \sum_{k=1}^{m}a_{ik}c_{kj}}$.

In addition, a Compressed Row Storage (CRS) sparse matrix data structure
and a function for matrix-vector multiplication using it are implemented.
In this format, three arrays are used: $val$, storing the nonzero values
of the matrix, $col\_idx$, storing the indices of these values, and $row\_ptr$,
storing which indices in the other two arrays correspond to a new row in the
matrix. The matrix-vector multiplication $Ax$ is then performed by iterating over
the $row\_ptr$ indices, multiplying and summing the nonzero values pointed to
by $col\_idx$ with the values of $x$ as described in Chapter 3.2 of the lecture notes.

# **Methods**

The approach I took when writing the implementations was to try to translate the
mathematical definitions as directly as possible. Using the function sum, the
inner product can be defined in the following way:

In [3]:
def innerprod(x, y):
    """
    Calculate the inner product (dot product) of vectors x and y.
    """
    return sum(x[i]*y[i] for i in range(len(x)))

The matrix-vector and matrix-matrix products can be calculated in different ways,
depending on whether the iteration should be performed row-wise or column-wise first,
which Golub and Van Loan (2013) describe in-depth. However, I base my implementations
on the definitions given in the introduction.

Using Python's indexing syntax, $a_j$ can be written as `A[:,j]`, which can be
combined with the function sum for a definition of the matrix-vector product
faithful to the original one:

In [4]:
def matvecprod(A, x):
    """
    Calculate the matrix-vector product of matrix A and vector y.
    """
    return sum(A[:,j]*x[j] for j in range(len(x)))

For the matrix-matrix product, we can observe that
${\displaystyle \sum_{k=1}^{m}a_{ik}c_{kj}} = (a^T_i,c_j)$
(where $a^T_i$ denotes the $i$th row of $A$). Reusing the inner
product function along with Python's list comprehension syntax, we get the following
definition:

In [5]:
def matmatprod(A, B):
    """
    Calculate the matrix-matrix product of matrices A and B.
    """
    return np.array([[innerprod(A[i,:],B[:,j]) for j in range(A.shape[0])] for i in range(B.shape[1])])

Finally, a CRS sparse matrix class was implemented. In addition to the three arrays
described in the introduction, the shape of the matrix is also stored, as the
original format does not necessarily convey the number of columns in the matrix.
A method performing matrix-vector multiplication belonging to the class was
implemented using Algorithm 1 of Chapter 3.2.

In [6]:
class SpMat:
    """
    A data structure to store sparse matrices in CRS format
    """
    def __init__(self, val, col_idx, row_ptr, shape):
        self.val = val
        self.col_idx = col_idx
        self.row_ptr = row_ptr
        self.shape = shape

    def spmatvecmul(self, x):
        """
        Calculate the matrix-vector product of sparse matrix self and vector x
        """
        n = self.shape[0]
        b = np.zeros(n)
        for i in range(n):
            for j in range(self.row_ptr[i],self.row_ptr[i+1]):
                b[i]+=self.val[j]*x[self.col_idx[j]]
        return b

# **Results**

The following cells present tests verifying the correctness of the functions written
in the section above. First, a test using data verified by hand is performed,
then the function is compared to the corresponding function in NumPy on a large set
of random data.

**Inner product**

First, I perform a test using vectors $x = (1,1,2)^T$ and $y=(3,1,-1)^T$. According
to the formula, the result should be $1\cdot 3 + 1 \cdot 2 + 2\cdot -1 = 2$.

In [7]:
x = np.array([1,1,2])
y = np.array([3,1,-1])
innerprod(x,y)

2

The following code tests the function using random data generated by the function
`np.random.rand`. The loop compares the result of 1000 function calls, treating the
result of innerprod as correct if it is within $10^{-6}$ of the result of `np.vdot`.

In [8]:
correct = 0
for i in range(1000):
    length = random.randint(100)
    x = random.rand(length)
    y = random.rand(length)
    correct += (abs(np.vdot(x,y)-innerprod(x,y)) < 1e-6)

print(correct, "/ 1000 correct")

1000 / 1000 correct


**Matrix-vector product**

I first perform the following test, where the matrix $A$ is a permutation matrix which
also flips the sign of the second component. Therefore, the output when multiplying
with the vector $x = (1, 1, 2)^T$ should be $(1, -2, 1)^T$.

In [9]:
A = np.array([[1, 0, 0],
              [0, 0, -1],
              [0, 1, 0]])
x = np.array([1,1,2])
matvecprod(A,x)

array([ 1, -2,  1])

The following code performs 1000 random tests in the same way as above, using
`np.linalg.norm` to measure the distance between the two solutions.

In [10]:
correct = 0
for i in range(1000):
    length = random.randint(100)
    x = random.rand(length)
    A = random.rand(random.randint(100), length)
    correct += (np.linalg.norm(np.matmul(A,x)-matvecprod(A,x)) < 1e-6)

print(correct, "/ 1000 correct")

1000 / 1000 correct


**Matrix-matrix product**

Here, I use two matrices representing rotations in the x-y-plane. A is a clockwise
planar rotation of $\pi/4$, and B is a clockwise planar rotation of $7\pi/4$. As
they make a full $2\pi$ together, the result when multiplying them should be the
identity.

In [11]:
A = np.array([[1, 0, 0],
              [0, cos(pi/4), sin(pi/4)],
              [0, -sin(pi/4), cos(pi/4)]])
B = np.array([[1, 0, 0],
              [0, cos(7*pi/4), sin(7*pi/4)],
              [0, -sin(7*pi/4), cos(7*pi/4)]])
matmatprod(A,B)

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00, -2.22044605e-16],
       [ 0.00000000e+00,  2.22044605e-16,  1.00000000e+00]])

Again, the same kind of random tests are performed.

In [12]:
correct = 0
for i in range(1000):
    rows= random.randint(30)
    cols= random.randint(30)
    A = random.rand(rows, cols)
    B = random.rand(cols, rows)
    correct += (np.linalg.norm(np.matmul(A,B)-matmatprod(A,B)) < 1e-6)

print(correct, "/ 1000 correct")

1000 / 1000 correct


**Sparse matrices**

To verify that the sparse matrix class works correctly, the example matrix
from the lecture notes is constructed, and its attributes printed.

In [13]:
A = SpMat([3,2,2,2,1,1,3,2,1,2,3],
          [0,1,3,1,2,2,2,3,4,4,5],
          [0,3,5,6,8,9,11],
          (6,6))
print("A.val: ", A.val)
print("A.col_idx: ", A.col_idx)
print("A.row_ptr: ", A.row_ptr)
print("A.shape: ", A.shape)

A.val:  [3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 3]
A.col_idx:  [0, 1, 3, 1, 2, 2, 2, 3, 4, 4, 5]
A.row_ptr:  [0, 3, 5, 6, 8, 9, 11]
A.shape:  (6, 6)


To verify the matrix-vector multiplication method, the same matrix $A$
is multiplied with the vector $x = (0,0,0,1,-1,0.5)^T$, which according
to calculations by hand should give the output $(2, 0, 0, 2, -1, -0.5)^T$

In [14]:
x = [0,0,0,1,-1,0.5]
A.spmatvecmul(x)

array([ 2. ,  0. ,  0. ,  2. , -1. , -0.5])

No random tests are performed for this method, as generating random sparse matrices
and converting them into the CRS format proved to be nontrivial.

# **Discussion**

The results show that the algorithms work correctly as far as the test data indicates,
which was to be expected considering that they are directly derived from well
established mathematical definitions.

# **References**

- Hoffman, J. 2018. *Introduction to Scientific Computing*
- Golub, Gene H. and Van Loan, Charles F. 2013. *Matrix Computations*. 4th ed. Baltimore: John Hopkins University Press.